In [0]:
# !pip install gensim
# !pip install tf-nightly-2.0-preview
# !pip install plotly-express


# !pip uninstall tensorflow==tf-nightly-2.0-preview

# !pip install tensorflow
# !pip uninstall tensor-hub==0.5.0


# !pip install tensorflow-hub==0.4.0
# !pip install tensorflow==1.9.0

In [0]:
from google.colab import drive
import gzip
drive.mount('/content/drive/')
__dir__ = '/content/drive/My Drive/W266_Project'

GZIP_FILE = __dir__ + '/data/GoogleNews-vectors-negative300.bin.gz'
# f = gzip.open(GZIP_FILE, 'r')
# DATA_FILE = __dir__ + 'data/GoogleNews-vectors-negative300.txt'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
EMBED_DIM = 300
MAX_SEQ = 26


In [0]:
import tensorflow as tf
# import tensorflow_hub as hub
import pandas as pd
import nltk
import gensim
import re, os, csv
from smart_open import open
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter, defaultdict
import pandas as pd
from itertools import chain

from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support,classification_report
from sklearn.preprocessing import LabelEncoder

from keras import preprocessing as prepro
from keras.layers import TimeDistributed, Dense, Activation, Input, Conv1D, Embedding, GlobalMaxPooling1D, Dropout
from keras.layers import concatenate, SimpleRNN, LSTM, MaxPooling1D
from keras.models import Model
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import metrics

from tensorflow.python.tools import inspect_checkpoint as chkp
%matplotlib inline
print(tf.__version__)
# print(hub.__version__)

1.14.0


Using TensorFlow backend.


In [0]:
print('This may take a few minutes..')
model = gensim.models.KeyedVectors.load_word2vec_format(GZIP_FILE, binary=True)
# model.save(word_embeddings)
# model.save_word2vec_format(DATA_FILE, binary=False)
print('Done!')

This may take a few minutes..


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Done!


In [0]:
def checkPrior(blah):
	if (blah.prior is None or blah.prior is np.NaN) and (blah.prior_pos is None or blah.prior_pos is np.NaN):
		return True
	else:
		return False


In [0]:
def create_onto_data_vectors(data_type):
  data_type_dict = {'test' : '/content/drive/My Drive/W266_Project/data/onto.test.ner',
                   'dev' : '/content/drive/My Drive/W266_Project/data/onto.development.ner',
                   'train' : '/content/drive/My Drive/W266_Project/data/onto.train.ner'}
  header = 0
  if 'onto' in data_type_dict[data_type]:
    header = None
  data = pd.read_csv(data_type_dict[data_type], sep="\t",  quoting=csv.QUOTE_NONE, header=header, skip_blank_lines=False, engine='python', names =['token', 'pos', 'tree', 'label'])

  # see if prior row was a newline
  data['prior'] = data.token.shift(1)
  data['prior_pos'] = data.pos.shift(1)
  # drop empty rows
  data = data.loc[~data.token.isnull()]
  data.prior = data.apply(checkPrior, axis=1)
  data['phrase'] = data.prior.cumsum()
  df = data.groupby('phrase').agg({'token':list, 'label': list})
  X = df['token'].tolist()
  y = df['label'].tolist()
  return X, y

X, y = create_onto_data_vectors('train')

X_dev, y_dev = create_onto_data_vectors('dev')

X_test, y_test = create_onto_data_vectors('test')

print(len(X))
print(len(y))




115812
115812


In [0]:
# X, y = numpy.array(X), numpy.array(y)
# # print(X[1])
# # print(y[1])
# data_len = [len(x) for x in X]
# print(sum(data_len)/len(X))
# fig = plotly.graph_objects.FigureWidget(data = plotly.graph_objects.Bar(data_len))
# fig.show()

def filter_data_seq(X):
  max_length = MAX_SEQ
  lengths = [len(x) for x in X]
  print(f'Maximum length: {max(lengths)}')
  print(f'Minimum length: {min(lengths)}')
  print(f'Average length: {sum(lengths)/len(lengths)}')
  
  short_sentences = [s for s in X if len(s) <= max_length]
  print(f'% of short sentences - {100 * len(short_sentences)/len(lengths)}%')

filter_data_seq(X)

Maximum length: 228
Minimum length: 1
Average length: 19.003652471246504
% of short sentences - 76.4808482713363%


In [0]:
corpus = [token for sentence in X for token in sentence]
vocab = ['<PAD>', '<UNK>'] + [token for token, count in Counter(corpus).items() if count >= 2]
print(f'Length of vocabulary - {len(vocab)}')
token2index = defaultdict(lambda: 1, {token: index for index, token in  enumerate(vocab)})





Length of vocabulary - 34670


In [0]:
embeddings = np.zeros((len(token2index.keys()), EMBED_DIM))
no_embed = []
print(embeddings.shape)

for k, v in token2index.items():
   try:
      embeddings[v] = model[k]
   except:
      no_embed.append((k,v))
      continue

(34670, 300)


In [0]:
# print(embeddings[2])
print(f'% of volcabulary with no embeddings - {100 * len(no_embed)/len(vocab)}%')

# print(model['of'])
# print(embeddings[4])
# np,random.randn(.25)

% of volcabulary with no embeddings - 8.269397173348716%


In [0]:
# df_vocab = pd.DataFrame(list(token2index.items()), columns=['token', 'indx'])



label_vocab = ['<PAD>'] + list(set([label for target_list in y for label in target_list]))

label2index = {label : index for index, label in enumerate(label_vocab)}

print(f'Length of labels - {len(label2index)}')

max_label = len(label2index)
# print(label2index)

Length of labels - 38


In [0]:
FILE_PATH = '/content/drive/My Drive/W266_Project/data/bios-tagged-final-flat.csv'

header = 0
if 'onto' in FILE_PATH:
	header = None
data = pd.read_csv(FILE_PATH, sep="\t",  quoting=csv.QUOTE_NONE, header=header, skip_blank_lines=False, engine='python', names =['token', 'pos', 'tree', 'label'])

# see if prior row was a newline
data['prior'] = data.token.shift(1)
data['prior_pos'] = data.pos.shift(1)
# drop empty rows
data = data.loc[~data.token.isnull()]
data.prior = data.apply(checkPrior, axis=1)
data['phrase'] = data.prior.cumsum()

#aggregate by sentence
bios_df = data.groupby('phrase').agg({'token':list, 'label': list})
X_bios = bios_df['token'].tolist()
y_bios = bios_df['label'].tolist()

y_bios_no_title = [['O' if label in ['B-TITLE','U-TITLE','L-TITLE','I-TITLE'] else label for label in seq] for seq in y_bios]

In [0]:
X_digitized = [[token2index[word] for word in seq] for seq in X] 
y_digitized = [[label2index[label] for label in seq] for seq in y]

X_dev_digitized = [[token2index[word] for word in seq] for seq in X_dev] 
y_dev_digitized = [[label2index[label] for label in seq] for seq in y_dev] 

X_test_digitized = [[token2index[word] for word in seq] for seq in X_test] 
y_test_digitized = [[label2index[label] for label in seq] for seq in y_test] 

X_bios_digitized = [[token2index[word] for word in seq] for seq in X_bios] 
y_bios_digitized = [[label2index[label] for label in seq] for seq in y_bios_no_title] 



y_digitized  = [to_categorical(target, max_label) for target in y_digitized]
y_dev_digitized  = [to_categorical(target, max_label) for target in y_dev_digitized]
y_test_digitized  = [to_categorical(target, max_label) for target in y_test_digitized]
y_bios_digitized  = [to_categorical(target, max_label) for target in y_bios_digitized]

In [0]:
X_padded = prepro.sequence.pad_sequences(X_digitized, maxlen=MAX_SEQ, padding='post', truncating='post', value=0.0)
y_padded = prepro.sequence.pad_sequences(y_digitized, maxlen=MAX_SEQ, padding='post', truncating='post', value=0.0)

X_dev_padded = prepro.sequence.pad_sequences(X_dev_digitized, maxlen=MAX_SEQ, padding='post', truncating='post', value=0.0)
y_dev_padded = prepro.sequence.pad_sequences(y_dev_digitized, maxlen=MAX_SEQ, padding='post', truncating='post', value=0.0)

X_test_padded = prepro.sequence.pad_sequences(X_test_digitized, maxlen=MAX_SEQ, padding='post', truncating='post', value=0.0)
y_test_padded = prepro.sequence.pad_sequences(y_test_digitized, maxlen=MAX_SEQ, padding='post', truncating='post', value=0.0)

X_bios_padded = prepro.sequence.pad_sequences(X_bios_digitized, maxlen=MAX_SEQ, padding='post', truncating='post', value=0.0)
y_bios_padded = prepro.sequence.pad_sequences(y_bios_digitized, maxlen=MAX_SEQ, padding='post', truncating='post', value=0.0)

In [0]:
print(X_digitized[0])
print(X_padded[0].shape)
print(y_padded[0].shape)

[2, 3, 4, 5, 6]
(26,)
(26, 38)


In [0]:
# x_model = tf.placeholder(tf.int32, shape=[None, MAX_SEQ])
# with tf.device("/cpu:0"):
#   embedded_x = tf.nn.embedding_lookup(embeddings, x_model)


In [0]:
del model


In [0]:
VOCAB_SIZE = len(vocab)
print(MAX_SEQ)
CONV_FILTER = [5000,300,500,400,500]
CONV_KERNEL = [5,4,3,2,1]
BATCH_SIZE = 400
EPOCHS = 5
TAG_SIZE = max_label
DROP_RATE = .2

26


In [0]:
l_inp = Input(shape=(MAX_SEQ,))
l_embed = Embedding(VOCAB_SIZE, EMBED_DIM, weights=[embeddings], trainable = True, input_length=MAX_SEQ)(l_inp)


l_conv = Conv1D(CONV_FILTER[0], CONV_KERNEL[0], padding='same', activation='relu')(l_embed)
l_conv = Conv1D(CONV_FILTER[1], CONV_KERNEL[1], padding='same', activation='relu')(l_conv)
# l_conv = Conv1D(CONV_FILTER[2], CONV_KERNEL[2], padding='same', activation='relu')(l_conv)
# l_conv = Conv1D(CONV_FILTER[3], CONV_KERNEL[3], padding='same', activation='relu')(l_conv)
# l_conv = Conv1D(CONV_FILTER[4], CONV_KERNEL[4], padding='same', activation='relu')(l_conv)

l_dense1 = Dense(256, activation='relu')(l_conv)
# l_dense2 = Dense(64, activation='relu')(l_dense1)
l_output = Dense(TAG_SIZE, activation='softmax')(l_dense1)


W0803 20:25:37.798830 139645022852992 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0803 20:25:37.863134 139645022852992 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0803 20:25:37.875210 139645022852992 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0803 20:25:37.895916 139645022852992 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0803 20:25:37.897227 1396450228

In [0]:
# keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
adam_customized = Adam(lr=0.0004, beta_1=0.85, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [0]:
model = Model(inputs=l_inp, outputs=l_output)
model.compile(loss='categorical_crossentropy', optimizer=adam_customized, metrics=['acc'])


W0803 20:25:45.350965 139645022852992 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 26)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 26, 300)           10401000  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 26, 5000)          7505000   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 26, 300)           6000300   
_________________________________________________________________
dense_1 (Dense)              (None, 26, 256)           77056     
_________________________________________________________________
dense_2 (Dense)              (None, 26, 38)            9766      
Total params: 23,993,122
Trainable params: 23,993,122
Non-trainable params: 0
________________________________________________________________

In [0]:
model.reset_states()
r = model.fit(X_padded, y_padded, epochs=EPOCHS, validation_data=(X_dev_padded, y_dev_padded), batch_size=BATCH_SIZE)

Train on 115812 samples, validate on 15680 samples
Epoch 1/5
115812/115812 [==============================] - 148s 1ms/step - loss: 0.0801 - acc: 0.6029 - val_loss: 0.0867 - val_acc: 0.6140
Epoch 2/5
115812/115812 [==============================] - 141s 1ms/step - loss: 0.0565 - acc: 0.6088 - val_loss: 0.0801 - val_acc: 0.6157
Epoch 3/5
115812/115812 [==============================] - 141s 1ms/step - loss: 0.0436 - acc: 0.6125 - val_loss: 0.0818 - val_acc: 0.6157
Epoch 4/5
115812/115812 [==============================] - 141s 1ms/step - loss: 0.0338 - acc: 0.6155 - val_loss: 0.0851 - val_acc: 0.6163
Epoch 5/5
115812/115812 [==============================] - 141s 1ms/step - loss: 0.0257 - acc: 0.6180 - val_loss: 0.0909 - val_acc: 0.6163


In [0]:
with open(__dir__+'/Model_Output/cnn_bio_architecture.json', 'w') as f:
    f.write(model.to_json())

model.save_weights(__dir__+'/Model_Output/cnn_bio_model_weights.h5')

In [0]:
def reshape_predictions(yh, pr):
    coords = [np.where(yhh > 0)[0][0] for yhh in yh]
    yh = [yhh[co:] for yhh, co in zip(yh, coords)]
    ypr = [prr[co:] for prr, co in zip(pr, coords)]
    fyh = [c for row in yh for c in row]
    fpr = [c for row in ypr for c in row]
    return fyh, fpr

In [0]:

y_pred = model.predict(X_padded).argmax(2)
yh = y_padded.argmax(2)
fyh, fpr = reshape_predictions(yh, y_pred)

In [0]:
print('Training accuracy:', accuracy_score(fyh, fpr))
# print('Training confusion matrix:')
cm = confusion_matrix(fyh, fpr)

Training accuracy: 0.6244500370627197


In [0]:
print(classification_report(fyh, fpr))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00   1125578
           1       0.98      0.98      0.98      1014
           2       0.98      0.95      0.97      1549
           3       1.00      0.94      0.97        16
           4       0.99      0.99      0.99     19922
           5       0.97      0.98      0.97      1477
           6       0.90      0.98      0.94      1906
           7       0.99      0.63      0.77       708
           8       0.98      0.97      0.97       289
           9       0.96      0.99      0.97     18148
          10       0.99      0.99      0.99     20890
          11       0.98      0.98      0.98       828
          12       0.99      0.99      0.99      4030
          13       0.99      0.99      0.99     26693
          14       0.99      0.99      0.99      7818
          15       0.98      0.99      0.98      3055
          16       0.97      0.99      0.98       514
          17       0.99    

In [0]:
y_pred_test = model.predict(X_test_padded).argmax(2)
yh_test = y_test_padded.argmax(2)
fyh_test, fpr_test = reshape_predictions(yh_test, y_pred_test)
print('Training accuracy:', accuracy_score(fyh_test, fpr_test))

Training accuracy: 0.5955446697854818


In [0]:
print(classification_report(fyh_test, fpr_test))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00    121404
           1       0.71      0.55      0.62       128
           2       0.68      0.44      0.53       178
           4       0.87      0.80      0.83      1939
           5       0.67      0.64      0.65       158
           6       0.54      0.55      0.55       235
           7       0.76      0.25      0.38        51
           8       0.75      0.32      0.44        19
           9       0.79      0.80      0.80      1833
          10       0.81      0.76      0.79      1692
          11       0.67      0.56      0.61        75
          12       0.91      0.90      0.90       455
          13       0.81      0.82      0.81      2196
          14       0.84      0.80      0.82       819
          15       0.77      0.67      0.71       318
          16       0.80      0.42      0.55       143
          17       0.59      0.28      0.38       121
          18       0.55    

In [0]:
y_pred_bios = model.predict(X_bios_padded).argmax(2)
yh_bios = y_bios_padded.argmax(2)
fyh_bios, fpr_bios = reshape_predictions(yh_bios, y_pred_bios)
print('Training accuracy:', accuracy_score(fyh_bios, fpr_bios))

Training accuracy: 0.5757279443720121


In [0]:
print(classification_report(fyh_bios, fpr_bios))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      5818
           2       0.00      0.00      0.00         0
           4       0.91      0.80      0.86       335
           5       0.00      0.00      0.00         0
           7       0.00      0.00      0.00        40
           9       0.81      0.56      0.66       406
          10       0.69      0.55      0.61       882
          11       0.00      0.00      0.00         0
          12       0.50      1.00      0.67         1
          13       0.72      0.69      0.71      1597
          14       0.14      0.33      0.20         3
          16       1.00      0.50      0.67         2
          18       0.00      0.00      0.00         0
          19       0.00      0.00      0.00         2
          20       0.68      0.85      0.76       171
          21       0.00      0.00      0.00         1
          22       0.78      0.71      0.74       436
          25       0.67    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
score = model.evaluate(X_bios_padded, y_bios_padded)
print(f'Model loss: {score}')

708/708 [==============================] - 1s 796us/step
Model loss: [0.614660323339667, 0.5757279448253286]
